In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [4]:
from keras.layers.core import Dense, Dropout, SpatialDropout1D
from keras.layers.convolutional import Conv1D
from keras.layers.embeddings import Embedding
from keras.layers.pooling import GlobalMaxPooling1D
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.callbacks import Callback, ModelCheckpoint
from keras.models import load_model
from sklearn.model_selection import train_test_split
import collections
import matplotlib.pyplot as plt
import nltk
import numpy as np
np.random.seed(42)

In [5]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [14]:
from gensim.models import KeyedVectors

In [6]:
train_name = 'input/train.csv'
test_name = 'input/test.csv'

In [7]:
train = pd.read_csv(train_name)

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
id               159571 non-null object
comment_text     159571 non-null object
toxic            159571 non-null int64
severe_toxic     159571 non-null int64
obscene          159571 non-null int64
threat           159571 non-null int64
insult           159571 non-null int64
identity_hate    159571 non-null int64
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [9]:
train.head(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [10]:
sub = pd.read_csv(test_name, )

In [11]:
sub.head(5)

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [12]:
target_names = list(train.columns[2:])
target_names

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [15]:
INPUT_FILE = "input/train.csv"
VOCAB_SIZE = 7500

In [16]:
counter = collections.Counter()
#fin = open(INPUT_FILE, "r", encoding = 'utf8')
maxlen = 0
line_sizes = []

for i in range(train.shape[0]):
    #clean a line
    line = train['comment_text'].values[i].replace('"', '').replace('\n', '').replace('\'', '')
    #make a list of word in a line (with purpose to know the maximum length of the line)
    words = [x.lower() for x in nltk.word_tokenize(line)]
    #line sizes
    line_sizes.append(len(words))
    if len(words) > maxlen:
        maxlen = len(words)
    for word in words:
        counter[word] += 1

In [17]:
word2index = collections.defaultdict(int)
for wid, word in enumerate(counter.most_common(VOCAB_SIZE)):
    #word[0] is the word itself and word[1] is its frequency in the file
    #so for word2index dictionary we only need word[0]
    word2index[word[0]] = wid + 1

In [18]:
len(counter)

302705

In [19]:
vocab_size = len(word2index) + 1
index2word = {v:k for k, v in word2index.items()}

In [20]:
maxlen

4948

In [21]:
line_sizes_arr = np.array(line_sizes)

In [27]:
#I decided to cut off the long sentences by 300 words
maxlen = 300

In [28]:
xs, ys = np.zeros((train.shape[0], maxlen)), np.zeros((train.shape[0], 6))

for i in range(train.shape[0]):
    #clean a line
    line = train['comment_text'].values[i].replace('"', '').replace('\n', '').replace('\'', '')
        
    words = [x.lower() for x in nltk.word_tokenize(line)][:maxlen]
    wids = [word2index[word] for word in words]
    wids_padded = wids + [0] * (maxlen - len(wids))
    xs[i] = np.array(wids_padded)
    
    ys[i] = train.iloc[i, 2:].values
    
    if i % 10000 == 0:
        print(i, end = ', ')

0, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, 110000, 120000, 130000, 140000, 150000, 

In [29]:
xs.shape

(159571, 300)

#### #xs_sub

In [30]:
xs_sub = np.zeros((sub.shape[0], maxlen))

for i in range(sub.shape[0]):
    #clean a line
    line_sub = sub['comment_text'].values[i].replace('"', '').replace('\n', '').replace('\'', '')
        
    words_sub = [x.lower() for x in nltk.word_tokenize(line_sub)][:maxlen]
    wids_sub = [word2index[word] for word in words_sub]
    wids_padded_sub = wids_sub + [0] * (maxlen - len(wids_sub))
    xs_sub[i] = np.array(wids_padded_sub)
    
    if i % 10000 == 0:
        print(i, end = ', ')

0, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, 110000, 120000, 130000, 140000, 150000, 

In [31]:
xs_sub.shape

(153164, 300)

##### #AUC - for Callback

In [32]:
from sklearn.metrics import roc_auc_score
class roc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]


    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.x)
        roc = roc_auc_score(self.y, y_pred)
        y_pred_val = self.model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,4)),str(round(roc_val,4))),end=100*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

#### #y0

In [33]:
y_sub = np.zeros((sub.shape[0], 6))
y_sub.shape

(153164, 6)

In [34]:
from sklearn.metrics import roc_auc_score
from tensorflow.contrib.metrics import streaming_auc
import tensorflow as tf

def roc_auc2(y_true, y_pred):
    a = roc_auc_score(y_true, tf.Session().run(y_pred))

In [35]:
WORD2VEC_MODEL = '../word_embeddings/GoogleNews-vectors-negative300.bin'

In [38]:
# load word2vec model
#gensim.models.KeyedVectors instead of gensim.models.Word2Vec (deprecated)
import gensim
word2vec = gensim.models.KeyedVectors.load_word2vec_format(WORD2VEC_MODEL, binary=True)

In [43]:
EMBED_SIZE = 300
embedding_weights = np.zeros((vocab_size, EMBED_SIZE))

In [44]:
for word, index in word2index.items():
    try:
        embedding_weights[index, :] = word2vec[word]
    except KeyError:
        pass

In [57]:
BATCH_SIZE = 512
NUM_EPOCHS = 5
NUM_FILTERS = 1024
NUM_WORDS = 4

for i in range(6):
    print('\nITERATION {}: {}\n'.format(i, target_names[i]))
    Y1 = np_utils.to_categorical(ys[:, i])
    #print('Y1.shape: {}'.format(Y1.shape))

    #print('Y1[:, 1].mean(): {}'.format(Y1[:, 1].mean()))

    Xtrain, Xtest, Ytrain, Ytest = train_test_split(xs, Y1, test_size=0.3, random_state=42)

    model = Sequential()
    model.add(Embedding(vocab_size, EMBED_SIZE, input_length=maxlen,
                       weights = [embedding_weights]))
    model.add(SpatialDropout1D(0.4))
    model.add(Conv1D(filters=NUM_FILTERS, kernel_size=NUM_WORDS, activation="relu"))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(2, activation="softmax"))
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
    checkpointer = ModelCheckpoint('models/model_{}.h5'.format(i), 
                                   monitor = 'val_acc',
                                   verbose=0, 
                                   save_best_only=True,
                                   mode = 'max')
    history = model.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE,
                        epochs=NUM_EPOCHS,
                        validation_data=(Xtest, Ytest),
                        callbacks=[checkpointer,
                                   roc_callback(training_data=(Xtrain, Ytrain),
                                                validation_data=(Xtest, Ytest))])

    model = load_model('models/model_{}.h5'.format(i))
    
    y_sub[:, i] = model.predict(xs_sub)[:, 1]


ITERATION 0: toxic

Train on 111699 samples, validate on 47872 samples
Epoch 1/5
111699/111699 [==============================] - 184s 2ms/step - loss: 0.1410 - acc: 0.9506 - val_loss: 0.1044 - val_acc: 0.9613
roc-auc: 0.9822 - roc-auc_val: 0.9754                                                                                                    
Epoch 2/5
111699/111699 [==============================] - 182s 2ms/step - loss: 0.0987 - acc: 0.9628 - val_loss: 0.1036 - val_acc: 0.9619
roc-auc: 0.9888 - roc-auc_val: 0.9767                                                                                                    
Epoch 3/5
111699/111699 [==============================] - 182s 2ms/step - loss: 0.0855 - acc: 0.9671 - val_loss: 0.1036 - val_acc: 0.9622
roc-auc: 0.9934 - roc-auc_val: 0.9757                                                                                                    
Epoch 4/5
111699/111699 [==============================] - 181s 2ms/step - loss: 0.0712 - acc: 0.

111699/111699 [==============================] - 182s 2ms/step - loss: 0.0133 - acc: 0.9954 - val_loss: 0.0286 - val_acc: 0.9921
roc-auc: 0.9987 - roc-auc_val: 0.9622                                                                                                    
Epoch 5/5
111699/111699 [==============================] - 181s 2ms/step - loss: 0.0102 - acc: 0.9964 - val_loss: 0.0321 - val_acc: 0.9914
roc-auc: 0.9994 - roc-auc_val: 0.9543                                                                                                    


In [58]:
y_sub

array([[9.99753177e-01, 1.60092175e-01, 9.95434701e-01, 6.82387650e-02,
        9.60392654e-01, 1.65845290e-01],
       [4.58791561e-04, 1.51148837e-04, 2.07091356e-03, 4.71603307e-05,
        1.75901374e-03, 1.11201825e-03],
       [4.62455899e-02, 1.37704052e-03, 1.65415835e-02, 1.34225465e-05,
        7.23181153e-03, 1.39323471e-03],
       ...,
       [5.52542333e-04, 2.93768331e-04, 9.83823789e-04, 2.01073926e-04,
        1.05217553e-03, 4.57630988e-04],
       [1.68083597e-03, 3.04245827e-04, 3.34065314e-03, 5.48638753e-04,
        4.57949936e-03, 2.83632129e-02],
       [9.77569163e-01, 1.24748365e-03, 6.66202664e-01, 3.14978277e-03,
        1.65339932e-01, 7.04401964e-03]])

In [60]:
sub1 = sub.copy()

In [61]:
for i, item in enumerate(target_names):
    sub1[item] = y_sub[:, i]

In [62]:
sub1.drop('comment_text', axis = 1).to_csv('output/sub11.csv', index = None)

In [63]:
sub1.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,0.999753,0.160092,0.995435,0.068239,0.960393,0.165845
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,0.000459,0.000151,0.002071,0.000047,0.001759,0.001112
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",0.046246,0.001377,0.016542,0.000013,0.007232,0.001393
3,00017563c3f7919a,":If you have a look back at the source, the in...",0.000190,0.000154,0.000687,0.000044,0.001032,0.000166
4,00017695ad8997eb,I don't anonymously edit articles at all.,0.014026,0.001095,0.004249,0.000468,0.009468,0.001160
5,0001ea8717f6de06,Thank you for understanding. I think very high...,0.000788,0.000133,0.000469,0.000233,0.001087,0.000229
6,00024115d4cbde0f,Please do not add nonsense to Wikipedia. Such ...,0.002254,0.000065,0.001079,0.000033,0.001222,0.000453
7,000247e83dcc1211,:Dear god this site is horrible.,0.494491,0.000782,0.017229,0.000520,0.016560,0.002629
8,00025358d4737918,""" \n Only a fool can believe in such numbers. ...",0.007183,0.000577,0.014373,0.001088,0.018738,0.000771
9,00026d1092fe71cc,== Double Redirects == \n\n When fixing double...,0.000122,0.000117,0.001747,0.000139,0.001440,0.000485
